# Compare Gap-Constrained Results with Valid Ensemble

In [ ]:
import pandas as pd
import numpy as np
from shapely import affinity
from shapely.geometry import Polygon
from itertools import combinations
import json

class ChristmasTree:
    def __init__(self, center_x, center_y, angle):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        
        initial_polygon = Polygon([
            (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5),
            (0.2, 0.25), (0.1, 0.25), (0.35, 0.0),
            (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2),
            (-0.075, 0.0), (-0.35, 0.0), (-0.1, 0.25),
            (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5),
        ])
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def get_bounding_box_side(trees):
    all_points = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_points.append(coords)
    all_points = np.vstack(all_points)
    min_x, min_y = all_points.min(axis=0)
    max_x, max_y = all_points.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def get_min_distance(trees):
    if len(trees) <= 1:
        return float('inf')
    min_dist = float('inf')
    for i, j in combinations(range(len(trees)), 2):
        dist = trees[i].polygon.distance(trees[j].polygon)
        min_dist = min(min_dist, dist)
    return min_dist

print("Functions defined")

In [ ]:
# Load valid ensemble and gap-constrained results
df_valid = pd.read_csv('/home/code/experiments/002_valid_submission/submission.csv')
df_gap = pd.read_csv('/home/code/experiments/006_true_gap_constrained/gap_constrained_result.csv')

print("Comparing N=1-20:")
print(f"{'N':>3} {'Valid':>10} {'Gap-SA':>10} {'Diff':>10} {'Valid min_d':>12} {'Gap min_d':>12}")

valid_total = 0
gap_total = 0

for n in range(1, 21):
    trees_valid = load_trees_for_n(df_valid, n)
    trees_gap = load_trees_for_n(df_gap, n)
    
    if len(trees_gap) != n:
        print(f"N={n}: Gap-SA missing")
        continue
    
    valid_side = get_bounding_box_side(trees_valid)
    gap_side = get_bounding_box_side(trees_gap)
    
    valid_contrib = (valid_side ** 2) / n
    gap_contrib = (gap_side ** 2) / n
    
    valid_min_d = get_min_distance(trees_valid)
    gap_min_d = get_min_distance(trees_gap)
    
    valid_total += valid_contrib
    gap_total += gap_contrib
    
    diff = gap_contrib - valid_contrib
    print(f"{n:3d} {valid_contrib:10.4f} {gap_contrib:10.4f} {diff:+10.4f} {valid_min_d:12.6f} {gap_min_d:12.6f}")

print(f"\nTotal N=1-20: Valid={valid_total:.4f}, Gap-SA={gap_total:.4f}, Diff={gap_total-valid_total:+.4f}")

In [ ]:
# The gap-constrained SA is much worse because it starts from random positions
# The valid ensemble is already well-optimized
# Let's just use the valid ensemble and log this experiment

# Copy valid ensemble to submission
import shutil
shutil.copy('/home/code/experiments/002_valid_submission/submission.csv', '/home/submission/submission.csv')
print("Using valid ensemble as submission")

# Save metrics
metrics = {'cv_score': 71.812779}
with open('/home/code/experiments/006_true_gap_constrained/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Metrics saved: {metrics}")